# 13.7 Authenticating with Twitter Via Tweepy to Access Twitter v2 APIs
* A **Tweepy `Client` object** is your gateway to using the Twitter v2 APIs
* Must first **authenticate with Twitter**

In [1]:
import tweepy

In [2]:
# before executing this cell, ensure that your copy of keys.py 
# contains your Twitter credentials as described earlier
import keys  

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Creating a `Client` Object

To use the Twitter v2 APIs, you must first create a `Client` object, initializing it with your bearer token: 

In [3]:
client = tweepy.Client(bearer_token=keys.bearer_token,
                       wait_on_rate_limit=True)

Two arguments:
* `bearer_token` is your bearer token 
* `wait_on_rate_limit=True` tells Tweepy that each time it reaches a given API method’s rate limit it should wait for the rate-limit interval to expire
    * This ensures that you do not violate Twitter’s rate-limit restrictions
    * For most Twitter APIs, the rate-limit interval is 15 minutes.

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

# 13.8 Getting Information About a Twitter Account
Use the Tweepy `Client` object’s `get_user` method to get a `tweepy.Response` object containing information about a @NASA’s Twitter account:

In [4]:
nasa = client.get_user(username='NASA',
    user_fields=['description', 'public_metrics'])

* `get_user` with the `username` keyword argument calls the Twitter API method 
> `/2/users/by/username/:username`
* Returns JSON data that Tweepy converts into a `tweepy.Response` 
* Twitter returns the account’s ID number, name and user name by default
* Can request additional user account fields via the `user_fields` keyword argument
    * We requested the account’s `description` and `public_metrics`
* Complete list of user fields can be viewed at:
> https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
* Each Twitter method has a rate limit
    * `/2/users/by/username/:username`
    * Can call this method up to 900 times every 15 minutes 
    

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### `tweepy.Response` Object
* Contains four fields:
    * `data` — contains the data returned by Twitter
    * `includes` — additional data specified via the method’s `expansions` parameter 
    * `errors` — information about any errors that occurred
    * `meta` — method-specific information that can be useful in processing the response

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Getting a User’s Basic Account Information
* When a Twitter method returns a **user JSON object**, `Response`’s `data` attribute is a named tuple containing the default fields:
    * `id` is the account’s unique ID number.
    * `name` is the name associated with the user’s account.
    * `username` is the user’s Twitter handle (`@NASA`) 
* Additional `user_fields` `description` and `public_metrics` (discussed momentarily) also are in the `Response` object’s `data` attribute

In [5]:
nasa.data.id

11348282

In [6]:
nasa.data.name

'NASA'

In [7]:
nasa.data.username

'NASA'

In [8]:
nasa.data.description

"There's space for everybody. ✨"

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Getting the Number of Accounts That Follow This Account and the Number of Accounts This Account Follows
* User account’s `public_metrics` are returned as a dictionary with keys:
    * `'followers_count'` — number of users who follow this account, 
    * `'following_count'` — e number of users that this account follows, 
    * `'tweet_count'` — total number of tweets (and retweets) sent by this user, and 
    * `'listed_count'` — total number of Twitter lists that include this user

In [9]:
nasa.data.public_metrics['followers_count']

62900622

In [10]:
nasa.data.public_metrics['following_count']

181

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Getting Your Own Account’s Information
* Get via `Client` object’s `get_me` method
> me = client.get_me()
* Returns a User object for the account you used to authenticate with Twitter

# 13.9 Intro to Tweepy `Paginator`s: Getting More than One Page of Results 
* Twitter API methods often return collections of objects, such as
    * tweets sent by a particular user, 
    * tweets matching specified search criteria or 
    * tweets in a user’s timeline (consisting of tweets sent by a user and by other accounts that user follows)

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

# 13.9 Intro to Tweepy `Paginator`s: Getting More than One Page of Results (cont.)
* Each Twitter API can return a maximum number of items per call
    * known as a page of results
* For more than one page of results, use a Tweepy `Paginator` to handle paging details
* `Paginator` invokes a specified `Client` method and checks whether there is another page of results
    * If so, the `Paginator` automatically calls the method again to get those results
    * Continues (subject to the method’s rate limits) until there are no more results to process

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

## 13.9.1 Determining an Account’s Followers  
* Use a `Paginator` to invoke the `Client` object’s `get_users_followers` method
* Calls the Twitter API’s method
> `/2/users/:id/followers`
* Returns these in groups of 100 by default
* Can request up to 1000 at a time
* We’ll grab 10 of NASA’s followers, five at a time, so we receive two pages of results
* Create a list to store the followers’ Twitter user names

In [11]:
followers = []

### Creating a `Paginator`
* `Paginator` to call `get_users_followers` for NASA’s account  

In [12]:
paginator = tweepy.Paginator(
   client.get_users_followers, nasa.data.id, max_results=5)

* arguments are the method to call and any arguments that should be passed to that method
    * `client.get_users_followers` indicates that the `Paginator` will call the `client` object’s `get_users_followers` method, 
    * `nasa.data.id` — ID number (obtained in Section 13.8) of the NASA Twitter account for which we’ll get followers, and 
    * `max_results=5` — results per page.

### Getting Results
* Use the `Paginator` to get some followers
    * `paginator.flatten(10)` initiates the call to `client.get_users_followers`
    * `10` indicates number of results to obtain

In [13]:
for follower in paginator.flatten(limit=10):
    followers.append(follower.username)

In [14]:
print('Followers:', 
    ' '.join(sorted(followers, key=lambda s: s.lower())))

Followers: AdamBracy83 AldairR38989312 BugginLol Carlosf91022160 enriqueLpezroj3 farzandadamizad lil_timmy44505 Lila_May22 MikazukSirius QP_SimonS


### Automatic Paging
* `flatten` automatically “pages” through the results by making multiple calls to `client.get_users_followers` as necessary
* `flatten` makes multiple pages appear to be a sequence of results
* If you do not specify an argument to `flatten`, the `Paginator` attempts to get all of the account’s followers
    * This could take significant time due to Twitter’s rate limits 
    * `/2/users/:id/followers` can return a maximum of 1000 followers at a time, and Twitter allows up to 15 calls every 15 minutes
    * 15,000 followers every 15 minutes using Twitter’s free APIs
    * At 60,000 followers per hour, it would take over 40 days to get all of NASA’s followers

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

## 13.9.2 Determining Whom an Account Follows 
* `Client` object’s `get_users_following` method calls the Twitter API’s 
`/2/users/:id/following` method to get a list of Twitter users an account follows
* Returns groups of 100 by default, but you can request up to 1000 at a time
* Can call this method up to 15 times every 15 minutes
* Get 10 accounts that NASA follows:

In [15]:
following = []

paginator = tweepy.Paginator(
    client.get_users_following, nasa.data.id, max_results=5)

for user_followed in paginator.flatten(limit=10):
    following.append(user_followed.username)

print('Following:', 
      ' '.join(sorted(following, key=lambda s: s.lower())))

Following: Astro_Ayers astro_berrios astro_deniz astro_matthias Astro_Pam astro_watkins JimFree NASA_Gateway NASASpaceSci v_wyche


<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

## 13.9.3 Getting a User’s Recent Tweets 
* `Client` method `get_users_tweets` returns a `tweepy.Response` containing tweets from a specified user
* Calls the Twitter API’s `/2/users/:id/tweets` method
* Returns the most recent 10 tweets but can between 5 and 100 at a time
* Can return only an account’s 3200 most recent tweets
* May call this method up to 1500 times every 15 minutes 

## 13.9.3 Getting a User’s Recent Tweets (cont.) 
* The `data` attribute of the `tweepy.Response` contains a list of the returned tweets
    * Each object in that list has a dictionary `data` attribute containing the keys `'id'` and `'text'` for each tweet’s unique ID and its text
* Display five tweets from the `@NASA` account using its ID number that we obtained previously: 

In [16]:
nasa_tweets = client.get_users_tweets(
     id=nasa.data.id, max_results=5)

for tweet in nasa_tweets.data:
    print(f"NASA: {tweet.data['text']}\n")

NASA: RT @NASAArtemis: .@NASA will hold a media teleconference at 12:30pm ET on Sept. 23 to discuss yesterday's cryogenic test for the #Artemis I…

NASA: @RollerRdr You can download the pictures in this gallery and have them printed. Scroll down for the link to full-resolution TIF and PNG versions of @NASAWebb images here: https://t.co/iyIetrnIxP

NASA: Today marks the September equinox – the #FirstDayOfFall in the Northern Hemisphere and the first day of spring in the Southern Hemisphere – which occurs when both hemispheres see equal amounts of daylight: https://t.co/tvxoSjONUf

How do you mark the changing seasons? https://t.co/pefGlH8Iyn

NASA: @lvictorfaulkner @KILAGRIM @JHUAPL It won't hit the asteroid hard enough to knock it out of orbit, but our goal is to change the length of that orbit. That demonstrates that we can change the course of an asteroid.

NASA: @carolineGx8 @JHUAPL Planetary defenders unite! Test your knowledge with our new and improved Planetary Defense quiz: http

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

## 13.9.3 Getting a User’s Recent Tweets (cont.)
* We called the `get_users_tweets` method directly and used the keyword argument `max_results` to specify the number of tweets to retrieve
* For more than the maximum number of tweets per call (100), use a `Paginator` to call `get_users_tweets` 

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Grabbing Recent Tweets from Your Own Timeline
* `Client` method `get_home_timeline` gets tweets from your home timeline
    * your tweets and retweets, as well as tweets and retweets from the Twitter users you follow
> `client.get_home_timeline()`
* Calls Twitter’s `/2/users/:id/timelines/reverse_chronological` method 
* Returns up to 100 tweets by default

# 13.10 Searching Recent Tweets; Intro to Twitter v2 API Search Operators 
* `Client` method `search_recent_tweets` 
    * Returns tweets from the last seven days that match a query string you provide
    * Calls Twitter method `/2/tweets/search/recent`, 
    * Returns a minimum of 10 tweets at a time (the default) but can return up to 100 (specified with keyword argument max_results)
    * It’s possible that fewer than 10 tweets will match the specified query string.

### Utility Function `print_tweets` from `tweetutilities.py`
* Receives the results of a call to API method `search` and for each tweet displays the user’s `screen_name` and the tweet’s `text`. 
* If the tweet is not in English and the `tweet.lang` is not `'und'` (undefined), we’ll also translate the tweet to English  

In [17]:
from tweetutilities import print_tweets

```python
def print_tweets(tweets):
    # translator to autodetect source language and return English
    translator = GoogleTranslator(source='auto', target='en')

    """For each tweet in tweets, display the username of the sender
    and tweet text. If the language is not English, translate the text 
    with the deep-translator library's GoogleTranslator."""
    for tweet, user in zip(tweets.data, tweets.includes['users']):
        print(f'{user.username}:', end=' ')

        if 'en' in tweet.lang:
            print(f'{tweet.text}\n')
        elif 'und' not in tweet.lang: # translate to English first
            print(f'\n  ORIGINAL: {tweet.text}')
            print(f'TRANSLATED: {translator.translate(tweet.text)}\n')
```

### Searching for Specific Words
* Call `Client` object’s `search_recent_tweets` method to search for 10 recent tweets about the Webb Space Telescope
* Returns a `Response` object in which the data attribute contains a list of matching tweets

In [18]:
tweets = client.search_recent_tweets(
    query='Webb Space Telescope', 
    expansions=['author_id'], tweet_fields=['lang']) 

* `query` keyword argument specifies the query string containing your search criteria
* Twitter returns only each tweet’s unique ID and text by default
* `'lang'` is an additional field you may request via the `tweet_fields` parameter
* Complete list of tweet fields
> https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet
* The expansion `'author_id'` indicates that for each tweet, Twitter also should return the user JSON object for the user who sent the tweet—`id`, `name` and `username` by default
* Tweepy places the expansion objects in the `Response`’s `includes` dictionary attribute
    * For the `'author_id'` expansion, a list of tweet authors is stored with the key `'users'`
    * Each tweet has a corresponding user in this list
    * The following expression in line 8 of `print_tweets` creates tuples in which the first element represents a tweet and the second element represents the user object for the sender
    > `zip(tweets.data, tweets.includes['users'])`

* Display the tweets 

In [19]:
print_tweets(tweets)

yoyo_eomer: RT @uhd2020: Dying Star Captured from the James Webb Space Telescope https://t.co/2DBQpEJKVX

Wara1329: RT @uhd2020: Jupiter from the James Webb Space Telescope https://t.co/VZP4hXOrmM

dwtpauline: RT @uhd2020: Jupiter from the James Webb Space Telescope https://t.co/VZP4hXOrmM

hemantK0303: RT @uhd2020: Dying Star Captured from the James Webb Space Telescope https://t.co/2DBQpEJKVX

Rosiebreeze: RT @nytimes: On Wednesday, the James Webb Space Telescope provided some of our best views of Neptune in 30 years.
https://t.co/ttBldkbW0l

rhuyshie: RT @uhd2020: Dying Star Captured from the James Webb Space Telescope https://t.co/2DBQpEJKVX

alanofkaneohe: RT @uhd2020: Jupiter from the James Webb Space Telescope https://t.co/VZP4hXOrmM

ValantisL: RT @uhd2020: Dying Star Captured from the James Webb Space Telescope https://t.co/2DBQpEJKVX

suuunantha: RT @uhd2020: Jupiter from the James Webb Space Telescope https://t.co/VZP4hXOrmM

steeldawn2012: RT @nbcwashington: NASA released n

### Searching with Twitter v2 API Search Operators
* Can use Twitter search operators in query strings to refine search results
* Max query-string length is limited by your developer account type:
    * For Essentials and Elevated accounts: up to 512 characters
    * For Academic Research accounts: up to 1024 characters
* Some operators are available only for Elevated accounts or higher
* The Twitter v2 operators are categorized as **standalone** or **conjunction-required**
    * **Standalone operators** can be used alone or combined with other operators in a query string
    * **Conjunction-required operators** must be combined with at least one standalone operator in a query string
* The following table shows several Twitter search operators, as well as logical AND, logical OR and logical negation capabilities
    * parentheses can be used to group query-string subexpressions
    * matching is performed using case-insensitive searching

| Example&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | Finds tweets containing |
| --- | --- |
| `python twitter` | Finds tweets containing `python` AND `twitter`. Spaces between query string terms and operators are implicitly treated as logical AND operations. In this query string, `python` and `twitter` are terms to search for—these are considered **standalone operators**.
| `python OR twitter` 	| Finds tweets containing `python` `OR` `twitter` `OR` both. The logical `OR` operator is case-sensitive.
| `planets -mars` 	| `-` (minus sign)—Finds tweets containing `planets` but not `mars`. The minus is the logical NOT operator and can be applied to any operator.
| An emoji | Use emojis as standalone operators to find tweets containing those emojis. 
| `has:hashtags`, `has:links`, `has:mentions`, `has:media`, … | You can combine these **conjunction-required operators** with standalone operators to find tweets containing hashtags, links, mentions of other users, media and more. 
| `is:retweet`, `is:reply`, `is:verified`, … | You can combine these **conjunction-required operators** with standalone operators to determine whether a tweet is a retweet, a tweet is a reply, the sender is a verified Twitter account and more. 
| `place:"New York City"` | Finds tweets that were sent near `"New York City"`. Multiword places should be quoted as shown here. 
| `from:NASA` 	| Finds tweets from the account `@NASA`.
| `to:NASA` 	| Finds tweets to the account `@NASA`. You also may use `to:id`, where `id` is the unique ID number of the user account.

### Operator Documentation and Tutorial
* All operators with examples of each  
> https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
* Twitter’s tutorial on building high-quality Twitter v2 API query strings to obtain the targeted results
> https://developer.twitter.com/en/docs/tutorials/building-high-quality-filters
* Twitter online tool to help you build Twitter v2 API query strings
> https://developer.twitter.com/apitools/query?query=

### Searching for Tweets From NASA Containing Links
* Use `from` and `has:links` operators to get recent tweets from `NASA` that contain hyperlinks

In [20]:
tweets = client.search_recent_tweets(
    query='from:NASA has:links', 
    expansions=['author_id'], tweet_fields=['lang'])

In [21]:
print_tweets(tweets)

NASA: RT @NASAArtemis: .@NASA will hold a media teleconference at 12:30pm ET on Sept. 23 to discuss yesterday's cryogenic test for the #Artemis I…



### Searching for a Hashtag
* Get tweets containing the hashtag `#metaverse`

In [22]:
tweets = client.search_recent_tweets(query='#metaverse', 
    expansions=['author_id'], tweet_fields=['lang'])

In [23]:
print_tweets(tweets)

kyusvl: RT @FantasyArenaNFT: @VitalikButerin thanks for every idea of the DAO, we are here developing thanks to you!  Check out this on @opensea ht…

Yuypenchawee82: RT @HyperNation8: 🚀Join us through these 3 citizen welfare! 
The HyperNation will be the one and only city you will live in! 🏙️

Follow @Hy…

Uditha25: RT @SomeFi_Network: 1/🔥#SOMEFI AIRDROP IS LIVE NOW🔥

All you need to do is complete extremely simple tasks to get amazing rewards.

❗️How t…

karugahvictor: 
  ORIGINAL: RT @Umair70603634: #nft #nftart #art #nfts #crypto #nftartist #digitalart #nftcommunity #nftcollector #cryptoart #ethereum #opensea #blockc…
TRANSLATED: RT @Umair70603634: #nft #nftart #art #nfts #crypto #nftartist #digitalart #nftcommunity #nftcollector #cryptoart #ethereum #opensea #blockc…

Nguyn7Quc: Join the Adamant Metanetwork Quests and claim your free rewards! https://t.co/z8qI1HIhQj  #adamantquests #adamantmetanetwork #metaverse

yunes14000: RT @HyperNation8: 🚀Join us through these 3 citizen welfar

# 13.11 Spotting Trends: Twitter Trends API
**Note: At the time of this writing, Twitter had not yet migrated their Trending Topics APIs from v1.1 to v2. The v1.1 APIs used in this section are accessible only to Twitter Developer accounts with “Elevated” access and higher.**

* If a topic **“goes viral,”** thousands or even millions of people could tweet about it
* Twitter calls these **trending topics** and maintains lists of them worldwide
* Via the Twitter v1.1 Trends API, you can get lists of locations with trending topics and lists of the top 50 trending topics for each location
* To use the v1.1 APIs in Tweepy, initialize an object of class `OAuth2BearerHandler` with your bearer token, then create an `API` object that uses the `OAuth2BearerHandler` object to authenticate with Twitter:

In [24]:
auth = tweepy.OAuth2BearerHandler(keys.bearer_token)

api = tweepy.API(auth=auth, wait_on_rate_limit=True)

## 13.11.1 Places with Trending Topics 
* Tweepy `API`’s **`available_trends` method** calls Twitter’s [`trends/available`](https://developer.twitter.com/en/docs/trends/locations-with-trending-topics/api-reference/get-trends-available)  
* Returns **list of dictionaries** representing locations 

In [25]:
available_trends = api.available_trends()

In [26]:
len(available_trends)

467

* Each element contains location’s `name`, `woeid` (**Yahoo! Where on Earth ID**) and more

In [27]:
available_trends[0]

{'name': 'Worldwide',
 'placeType': {'code': 19, 'name': 'Supername'},
 'url': 'http://where.yahooapis.com/v1/place/1',
 'parentid': 0,
 'country': '',
 'woeid': 1,
 'countryCode': None}

In [28]:
available_trends[1]

{'name': 'Winnipeg',
 'placeType': {'code': 7, 'name': 'Town'},
 'url': 'http://where.yahooapis.com/v1/place/2972',
 'parentid': 23424775,
 'country': 'Canada',
 'woeid': 2972,
 'countryCode': 'CA'}

## 13.11.1 Places with Trending Topics (cont.)
* WOEID 1 represents **worldwide** 
* WOEID values for several landmarks, cities, states and continents

| Place | WOEID | Place | WOEID |
| --- | --- | --- | --- |
| Statue of Liberty | 23617050 | Iguazu Falls | 468785
| Los Angeles, CA | 2442047 | United States | 23424977
| Washington, D.C. | 2514815 | North America | 24865672
| Paris, France | 615702 | Europe | 24865675

* Also can search for locations close to a **latitude** and **longitude** via the **Tweepy `API`’s `closest_trends` method**
* Calls Twitter's [`trends/closest` method](https://developer.twitter.com/en/docs/trends/locations-with-trending-topics/api-reference/get-trends-closest)

## 13.11.2 Getting a List of Trending Topics 
* Via Tweepy `API`’s **`get_place_trends` method** 
* Calls **Twitter Trends API’s [`trends/place` method](https://developer.twitter.com/en/docs/trends/trends-for-location/api-reference/get-trends-place)**
* Returns top 50 trending topics for the location 
* [Look up WOEIDs](http://www.woeidlookup.com) 
* Look up WOEID’s programmatically using **Yahoo!’s web services** via [Python libraries like `woeid`](https://github.com/Ray-SunR/woeid)

### Worldwide Trending Topics 

In [29]:
world_trends = api.get_place_trends(id=1)  # list containing one dictionary

* **`'trends'` key** refers to a list of dictionaries representing each trend

In [30]:
trends_list = world_trends[0]['trends']

* Each trend has **`name`**, **`url`**, **`promoted_content`** (whether it's an advertisement), **`query`** and **`tweet_volume`** keys

In [31]:
trends_list[0]

{'name': '村田兆治',
 'url': 'http://twitter.com/search?q=%E6%9D%91%E7%94%B0%E5%85%86%E6%B2%BB',
 'promoted_content': None,
 'query': '%E6%9D%91%E7%94%B0%E5%85%86%E6%B2%BB',
 'tweet_volume': 18966}

### Get Today's Worldwide Trending Topics (cont.)
* For **trends with more than 10,000 tweets**, the `tweet_volume` is the number of tweets; otherwise, it’s `None`
* Filter the list so that it contains only trends with more than 10,000 tweets:

In [32]:
trends_list = [t for t in trends_list if t['tweet_volume']]

* Sort the trends in _descending_ order by `tweet_volume`:

In [33]:
from operator import itemgetter 

In [34]:
trends_list.sort(key=itemgetter('tweet_volume'), reverse=True) 

### Get Today's Worldwide Trending Topics (cont.)
* Display names of the **top five trending topics**

In [35]:
for trend in trends_list[:5]:
    print(trend['name'])

Browns
Steelers
#一緒に虹5thNT
Stormzy
秋分の日


### New York City Trending Topics (WOEID `2459115`)

In [36]:
nyc_trends = api.get_place_trends(id=2459115)  # New York City WOEID

In [37]:
nyc_list = nyc_trends[0]['trends']

In [38]:
nyc_list = [t for t in nyc_list if t['tweet_volume']]

In [39]:
nyc_list.sort(key=itemgetter('tweet_volume'), reverse=True) 

In [40]:
for trend in nyc_list[:5]:
    print(trend['name'])

judge
Browns
Steelers
Dahmer
Bond


## 13.11.3 Create a Word Cloud from Trending Topics
* Visualize New York City’s trending topics with more than 10,000 tweets each

In [41]:
topics = {}  # dictionary to store trend names and volumes 

In [42]:
for trend in nyc_list:
    topics[trend['name']] = trend['tweet_volume']

## 13.11.3 Create a Word Cloud from Trending Topics (cont.)
* `prefer_horizontal=0.5` **suggests** that 50% of the words should be horizontal, but may ignore to fit content

In [43]:
from wordcloud import WordCloud

In [44]:
wordcloud = WordCloud(width=1600, height=900,
    prefer_horizontal=0.5, min_font_size=10, colormap='prism', 
    background_color='white')

In [45]:
wordcloud = wordcloud.fit_words(topics)

In [46]:
wordcloud = wordcloud.to_file('TrendingTwitter.png')

* **Instructor note: You might need to duplicate the cell below (select it in the margin then press `c` to copy it and `v` to paste it), then execute the cell to see the new word cloud**
* Depending on the number of trending topics, the word cloud may be sparse

![A word cloud generated from trending Twitter hashtags](./TrendingTwitter.png "A word cloud generated from trending Twitter hashtags")

------
&copy;1992&ndash;2020 by Pearson Education, Inc. All Rights Reserved. This content is based on Chapter 5 of the book [**Intro to Python for Computer Science and Data Science: Learning to Program with AI, Big Data and the Cloud**](https://amzn.to/2VvdnxE).

DISCLAIMER: The authors and publisher of this book have used their 
best efforts in preparing the book. These efforts include the 
development, research, and testing of the theories and programs 
to determine their effectiveness. The authors and publisher make 
no warranty of any kind, expressed or implied, with regard to these 
programs or to the documentation contained in these books. The authors 
and publisher shall not be liable in any event for incidental or 
consequential damages in connection with, or arising out of, the 
furnishing, performance, or use of these programs.                  